In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('CLEAN_EDIT_model_data.csv')

In [3]:
df = df.drop(df.columns.to_series()["Unnamed: 0"], axis=1)

In [4]:
y = df.drop(df.columns.to_series()["retweet_from": "zoo"], axis=1)
y.head()

,signal_x
0,stay
1,stay
2,stay
3,stay
4,stay


In [5]:
y = pd.get_dummies(y).values

In [6]:
y

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [7]:
x = df.drop(df.columns.to_series()["signal_x"], axis=1)
x.head()

,retweet_from,tweet_length,encoded_sentiment,polarity,tweet,ability,able,aboard,abort,aborted,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,137,105,1,0.366667,assuming acceleration of to but in a comfortab...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,64,113,1,0.200000,is capable of transporting satellite to orbit ...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,137,6,0,0.000000,yup,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,137,7,0,0.000000,part,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,137,96,1,0.650000,fly to most place on earth in under min and an...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
sentiment_data = x.drop(x.columns.to_series()["ability": "zoo"], axis=1)

In [9]:
tfidf_data = x.drop(x.columns.to_series()["retweet_from": "tweet"], axis=1)

In [16]:
tfidf_data.head()

,ability,able,aboard,abort,aborted,about,above,absolutely,absorb,absorbed,...,you,your,yours,yourself,yr,yup,zero,zip,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.decomposition import PCA

In [11]:
pca = PCA(n_components=50)

In [12]:
pca.fit(tfidf_data)

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [13]:
x_pca = pca.transform(tfidf_data)
x_pca

array([[-0.0305343 , -0.02636286, -0.00968375, ...,  0.03339166,
        -0.01949395,  0.08943085],
       [-0.05897778, -0.11300545,  0.09691305, ...,  0.05695217,
        -0.0010584 , -0.04488428],
       [-0.00220385,  0.04452375, -0.08819971, ..., -0.09189849,
         0.00892563, -0.01670351],
       ...,
       [-0.01966202, -0.00473765, -0.04375936, ...,  0.08182616,
        -0.01691029,  0.00881787],
       [-0.02563251, -0.00984365, -0.01820948, ...,  0.02368496,
        -0.02081571, -0.04801973],
       [-0.02191234, -0.00817561, -0.03202871, ..., -0.05192119,
         0.00925327,  0.02004722]])

In [1]:
#tfidf ANN

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_pca, y, test_size = 0.2, random_state = 0)

In [ ]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1, validation_data=(X_test, y_test))

Train on 1761 samples, validate on 441 samples
Epoch 1/100


In [15]:
#sentiment ANN

In [21]:
x = tfidf_data.values
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
classifier = Sequential()
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 15, kernel_initializer='uniform', activation = 'relu'))
classifier.add(Dense(units = 3, kernel_initializer='uniform', activation = 'softmax'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, validation_split=0.1, validation_data=(X_test, y_test))

Train on 1761 samples, validate on 441 samples
Epoch 1/100
